## Transformers Hugging Face Model

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 61 kB 474 kB/s 
     |████████████████████████████████| 3.3 MB 47.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


## Data Prepareing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import nltk
import numpy as np
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from ast import literal_eval
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IM_FINAL/feature_extraction_pubg.csv',sep = '\t')
# if need to transform pandas series back to original type
position_list = [literal_eval(dic) for dic in df_test['feature_positions']]
print(position_list[0])

{1: 0, 3: 4}


In [ ]:
W = 5
position_list = [literal_eval(dic) for dic in df_test['feature_positions']]
reviews = df_test['review'].tolist()
review_windows = dict()

In [ ]:
# W = 3
# review_windows_3 = dict()

In [ ]:
for review_idx in range(len(reviews)):
    if review_idx % 10000 == 0:
        print(review_idx)
    review = reviews[review_idx]
    tokens = nltk.word_tokenize(review)
    feature_pos = position_list[review_idx]
    sentiment_windows = []
    for token_index, category in feature_pos.items():
        if token_index <= W:
            candidate = " ".join(tokens[:token_index+W+1])
        elif token_index+W > len(tokens):
            candidate = " ".join(tokens[token_index-W:])
        else:
            candidate = " ".join(tokens[token_index-W:token_index+W+1])
        sentiment_windows.append((candidate,category))
    review_windows[review_idx] = sentiment_windows

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


## Model 2

In [ ]:
from transformers import pipeline
sentiment_analysis2 = pipeline("sentiment-analysis",model="finiteautomata/bertweet-base-sentiment-analysis")
print(sentiment_analysis2("I love this!"))

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


[{'label': 'POS', 'score': 0.9915593266487122}]


In [ ]:
sentiment_results = dict()

for review_id in range(len(reviews)):
  if review_id % 1000 == 0:
    print(review_id)
  sent_cat = list()
  review = reviews[review_id]
  position = position_list[review_id]
  review_window = review_windows[review_id]
  for i, sent_tuple in enumerate(review_window):
    line = sent_tuple[0]
    category = sent_tuple[1]
    try:
      sent = sentiment_analysis2(line)
    except IndexError:
      print(IndexError)
      continue
    sent_cat.append((category, sent))
  sentiment_results[review_id] = sent_cat

0
1000
2000
3000


Token indices sequence length is longer than the specified maximum sequence length for this model (142 > 128). Running this sequence through the model will result in indexing errors


<class 'IndexError'>
4000
<class 'IndexError'>
5000
<class 'IndexError'>
6000
7000
<class 'IndexError'>
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [ ]:
len(sentiment_results)

11567

In [ ]:
import pickle
a_file = open("sentiment_category_11566_.pkl", "wb")
pickle.dump(sentiment_results, a_file)
a_file.close()

In [ ]:
sentiment_results

{0: [(0, [{'label': 'NEG', 'score': 0.9808561205863953}]),
  (4, [{'label': 'NEG', 'score': 0.9837950468063354}])],
 1: [(0, [{'label': 'NEG', 'score': 0.8598531484603882}]),
  (0, [{'label': 'NEU', 'score': 0.9614514112472534}])],
 2: [(2, [{'label': 'NEG', 'score': 0.9762983322143555}]),
  (0, [{'label': 'NEG', 'score': 0.8621987104415894}]),
  (1, [{'label': 'NEG', 'score': 0.8253354430198669}]),
  (1, [{'label': 'NEG', 'score': 0.7583006024360657}])]}